In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv1D,
    MaxPooling1D,
    LSTM,
    Bidirectional,
    Dropout,
    Flatten,
    Dense,
    Input,
)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics

from utils import load_dataset, train, auc, plot_roc
from einops import rearrange

In [ ]:
NUM_OUTPUTS = 31

model = Sequential(
    [
        Input(shape=(1000, 4)),
        Conv1D(320, kernel_size=8, activation="relu"),
        MaxPooling1D(pool_size=4, strides=4),
        Dropout(0.2),
        Conv1D(480, kernel_size=8, activation="relu"),
        MaxPooling1D(pool_size=8, strides=8),
        Dropout(0.2),
        Flatten(data_format="channels_first"),
        Dense(512, activation="relu"),
        Dense(NUM_OUTPUTS, activation="softmax"),
    ],
    name="deepsea_full",
)

model.summary()

In [ ]:
es = EarlyStopping(monitor="val_loss", patience=10)

epochs = 250
validation_freq = 1
file = "FOX_3000"
X_train, y_train, X_test, y_test = load_dataset(file, labels="binlabels")
X_train = rearrange(X_train, "w h c -> w c h")
X_test = rearrange(X_test, "w h c -> w c h")

In [ ]:
model.compile(optimizer=Adam(lr=1e-3), loss="binary_crossentropy", metrics=["acc"])
# convert from shape (n, 4, 1000) to (n, 1000, 4)

model = train(
    dataset=(X_train, y_train, X_test, y_test),
    model=model,
    epochs=epochs,
    verbose=2,
    validation_freq=validation_freq,
    callbacks=[],
    batch_size=100,
)

y_pred = model.predict(X_test)
plot_roc(y_test, y_pred)
print(auc(y_test, y_pred))

In [ ]:
model.save(f"../model_saves/evals/{model.name}-{n}000.h5")